##Итоговая проектная работа на тему:Система рекомендаций по фильмам, основанная на исходящих ссылках из Википедии
   ### Выполнила:Фазылова Алика Виаленовна


In [1]:
import pandas as pd
import numpy as np
import json
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import re
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import issparse

In [2]:
# Загрузка данных
data = []
with open('/content/wp_movies_10k.ndjson') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)
df.columns = ['Title', 'Metadata', 'Links', 'Ratings1', 'Ratings2']

In [3]:
df

,Title,Metadata,Links,Ratings1,Ratings2
0,Deadpool (film),"{'image': 'Deadpool poster.jpg', 'name': 'Dead...","[Tim Miller (director), Simon Kinberg, Ryan Re...",84%,6.9/10
1,The Revenant (2015 film),"{'image': 'The Revenant 2015 film poster.jpg',...","[Alejandro González Iñárritu, Arnon Milchan, S...",82%,7.9/10
2,Suicide Squad (film),"{'image': 'Suicide Squad (film) Poster.png', '...","[David Ayer, Charles Roven, Richard Suckle, Wi...",26%,4.7/10
3,Spectre (2015 film),"{'image': 'spectre poster.jpg', 'name': 'Spect...","[Sam Mendes, Michael G. Wilson, Barbara Brocco...",65%,
4,Rebel Without a Cause,"{'distributor': 'Warner Bros.', 'image': 'Rebe...","[Nicholas Ray, David Weisbart, Stewart Stern, ...",96%,
...,...,...,...,...,...
9995,Prelude to a Kiss (film),"{'image': 'PreludeKissPoster.jpg', 'name': 'Pr...","[Norman René, Michael Gruskoff, Craig Lucas, A...",61%,
9996,The Black Balloon (film),{'image': 'TheBlackBalloon_Official-Poster.jpg...,"[Elissa Down, Jimmy Jack, Rhys Wakefield, Luke...",88%,
9997,Freelancers (film),"{'image': 'Freelancers (film).jpg', 'name': 'F...","[Jessy Terrero, Mark Canton, 50 Cent, Cheetah ...",None,None
9998,Difret,"{'image': 'Difret poster.jpg', 'name': 'Difret...","[Dave Eggar, 2014 Sundance Film Festival, The ...",None,None


In [4]:
df.describe()

,Title,Metadata,Links,Ratings1,Ratings2
count,10000,10000,10000,5584,5584
unique,10000,9999,10000,102,85
top,Jug Face,"{'image': 'Blade Runner poster.jpg', 'name': '...","[Sean Bridgers, Lauren Ashley Carter, Larry Fe...",100%,
freq,1,2,1,165,3722


In [5]:
df.isnull().sum() + (df == '').sum()

,0
Title,0
Metadata,0
Links,0
Ratings1,4416
Ratings2,8138


*Отсутсвует больше, или же почти половина оценок, пользоваться этими столбцами не будем*

Для анализа будем использовать только столбец с ссылками



In [6]:
# Проверим на дубликаты названий
df['Title'].duplicated().sum()

0

In [7]:
# Проверим на дубликаты ссылок
df['Links'].duplicated().sum()

0

In [8]:
# Получаем текст ссылок для каждого фильма
df['link_text'] = df['Links'].apply(lambda links: ' '.join(links))
df['link_text']

,link_text
0,Tim Miller (director) Simon Kinberg Ryan Reyno...
1,Alejandro González Iñárritu Arnon Milchan Stev...
2,David Ayer Charles Roven Richard Suckle Will S...
3,Sam Mendes Michael G. Wilson Barbara Broccoli ...
4,Nicholas Ray David Weisbart Stewart Stern Irvi...
...,...
9995,Norman René Michael Gruskoff Craig Lucas Alec ...
9996,Elissa Down Jimmy Jack Rhys Wakefield Luke For...
9997,Jessy Terrero Mark Canton 50 Cent Cheetah Visi...
9998,Dave Eggar 2014 Sundance Film Festival The Num...


In [9]:
# Загрузка стоп-слов из  nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# предобработка текста
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Удаление спец символов
    return text.lower()  # Приведение к нижнему регистру

df['link_text_clean'] = df['link_text'].apply(lambda text: clean_text(text))

# Удаляем стоп-слова
df['link_text_clean'] = df['link_text_clean'].apply(
    lambda text: ' '.join([word for word in text.split() if word not in stop_words])
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


*В наших данных много имен и фамилий актеров и режиссеров,такие данные лучше обрабатывать используя биграммы*

In [10]:
from gensim.models.phrases import Phrases, Phraser

# Создание биграмм
def generate_bigrams(texts, min_count=5, threshold=10):
    # Разделение текста на слова для обучения модели биграмм
    sentences = [text.split() for text in texts]

    # Обучаем биграммы
    bigram = Phrases(sentences, min_count=min_count, threshold=threshold)
    bigram_phraser = Phraser(bigram)

    # Применяем биграммы к текстам
    texts_with_bigrams = [' '.join(bigram_phraser[sentence]) for sentence in sentences]

    return texts_with_bigrams

# Применяем функцию создания биграмм к очищенным текстам
df['link_text_clean'] = generate_bigrams(df['link_text_clean'])

In [11]:
df['link_text_clean']

,link_text_clean
0,tim miller director simon_kinberg ryan_reynold...
1,alejandro_gonzlez irritu arnon_milchan steve_g...
2,david_ayer charles_roven richard_suckle smith ...
3,sam_mendes michael g_wilson barbara_broccoli j...
4,nicholas_ray david_weisbart stewart stern irvi...
...,...
9995,norman ren michael gruskoff craig lucas alec_b...
9996,elissa jimmy jack rhys_wakefield luke_ford act...
9997,jessy terrero mark_canton 50_cent cheetah_visi...
9998,dave eggar 2014_sundance film_festival numbers...


# Рассмотрим следующие способы получения эмбеддингов:TF-IDF,Word2Vec,BERT

# TF-IDF

*TF-IDF вычисляет важность каждого слова в документе относительно количества его употреблений в данном документе и во всей коллекции текстов*

In [12]:
# Обучаем TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
tf_X = tfidf.fit_transform(df['link_text_clean'])

In [13]:
# функция рекомендаций
def get_recommendations(titles, df, X, classifier_func, n=5):
    all_recs = []  # Список для хранения всех рекомендаций

    for title in titles:
        if title in df['Title'].values:
            idx = df.index[df['Title'] == title].tolist()[0]  #индекс фильма

            # Получаем рекомендации от выбранного классификатора
            similar_indices, similar_scores = classifier_func(X, idx, n)

            similar_titles = df['Title'].iloc[similar_indices] # названия похожих фильмов по индексам

            for similar_title, score in zip(similar_titles, similar_scores):
                all_recs.append({
                    'Original': title,
                    'Recommended': similar_title,
                    'Similarity': score
                })
        else:
            print(f"Фильм '{title}' не найден.")

    return pd.DataFrame(all_recs)

In [14]:
# Вектор входных фильмов
movie_titles = ["Edward Scissorhands", "La La Land (film)", "Amelia (film)", "Deadpool (film)", "The Intouchables"]


Для определения похожести фильмов будем использовать косинусное сходство.


---


*Косинусное сходство — это мера, которая используется для определения сходства между двумя векторами в пространстве. Оно вычисляется как косинус угла между этими векторами и позволяет оценить, насколько они направлены в одну сторону, независимо от их длины.*

In [15]:
# Косинусное сходство
def cosine_classifier(X, idx, n):
    cosine_sim = cosine_similarity(X[idx].reshape(1, -1), X).flatten()
    similar_indices = cosine_sim.argsort()[-n-1:-1][::-1]
    return similar_indices, cosine_sim[similar_indices]

In [16]:
print(get_recommendations(movie_titles, df, tf_X, cosine_classifier))

               Original                              Recommended  Similarity
0   Edward Scissorhands                           Batman Returns    0.363087
1   Edward Scissorhands                           Ed Wood (film)    0.328485
2   Edward Scissorhands                   Dick Tracy (1990 film)    0.319472
3   Edward Scissorhands           The Nightmare Before Christmas    0.305394
4   Edward Scissorhands           Planet of the Apes (2001 film)    0.295918
5     La La Land (film)             Manchester by the Sea (film)    0.425667
6     La La Land (film)                             Carol (film)    0.374311
7     La La Land (film)                      Chicago (2002 film)    0.362710
8     La La Land (film)                     Whiplash (2014 film)    0.362129
9     La La Land (film)               Lost in Translation (film)    0.358361
10        Amelia (film)                      The Next Karate Kid    0.292141
11        Amelia (film)                              The Reaping    0.291486

В качестве метрики для одного фильма возьмем Precision@K *Precision@k= количество релевантных (похожих) фильмов в первых k рекомендациях/k*

А затем возьмем среднее Precision (разделив сумму Precision@K, для каждого фильма выбранного для эксперимента, на количество фильмов, выбранных для эксперимента. )

**Для TF-IDF Средняя Precision@5=((3/5)+(5/5)+(0/5)+(5/5)+(2/5))/5=0,6**

Для уменьшения размерности и дальнейшей визуализации воспользуемся PCA.


---


*PCA (Principal Component Analysis, метод главных компонент) — это метод снижения размерности данных. Он позволяет преобразовать многомерные данные в пространство меньшей размерности, сохраняя при этом как можно больше информации из исходных данных.*

In [17]:
# Функция для применения PCA
def apply_pca(X):
    pca = PCA(n_components=2)
    X_embedded = pca.fit_transform(X.toarray())
    return X_embedded


In [18]:
# Функция для визуализации кластеров
def visualize_clusters(X_embedded, df, n_clusters=100):
    # Кластеризация с K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['Cluster'] = kmeans.fit_predict(X_embedded)

    # Создаем DataFrame для визуализации
    emb_df = pd.DataFrame(X_embedded, columns=['x', 'y'])
    emb_df['Title'] = df['Title']
    emb_df['Cluster'] = df['Cluster']

    # Визуализация
    fig = px.scatter(emb_df, x='x', y='y', color='Cluster',
                     title='Кластеры фильмов',
                     labels={'x': 'PCA 1', 'y': 'PCA 2'},
                     hover_name='Title', color_continuous_scale=px.colors.sequential.Viridis)

    # Отображение графика
    fig.show()

In [19]:
# PCA
tf_X_embedded = apply_pca(tf_X)

visualize_clusters(tf_X_embedded, df)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Данные явно разделяются на два кластера, индийское и западное кино

Попробует ранжировать данные на обработанных PCA эмбеддингах

In [20]:
print(get_recommendations(movie_titles, df,tf_X_embedded,cosine_classifier))

               Original                               Recommended  Similarity
0   Edward Scissorhands                      Victoria (2015 film)    1.000000
1   Edward Scissorhands                                     Borat    1.000000
2   Edward Scissorhands               Waterloo Bridge (1940 film)    1.000000
3   Edward Scissorhands                             Inchon (film)    0.999999
4   Edward Scissorhands               Emma (1996 theatrical film)    0.999999
5     La La Land (film)                        On the Town (film)    1.000000
6     La La Land (film)                           Girlhood (film)    1.000000
7     La La Land (film)                  The Mirror Has Two Faces    1.000000
8     La La Land (film)                         Beverly Hills Cop    0.999999
9     La La Land (film)                               Juno (film)    0.999999
10        Amelia (film)                        Heaven (2002 film)    1.000000
11        Amelia (film)                       Shrek Forever Afte

**Для TF-IDF + PCA: Средняя Precision@5=((0/5)+(2/5)+(0/5)+(2/5)+(0/5))/5=0,16**

Качество сильно ухудшилось, не будем применять PCA для классификации,оставим только для визуализации

# Word2Vec

*Word2Vec — это популярная модель обучения вложений слов, предложенная исследователями Google в 2013 году (Томас Миколов). Она позволяет преобразовать слова из корпуса текстов в векторы чисел таким образом, что слова с похожими семантическими значениями имеют близкие векторные представления в многомерном пространстве.*

In [21]:
from gensim.models import Word2Vec

In [22]:
# Подготавливаем данные для Word2Vec
sentences = [text.split() for text in df['link_text_clean']]

In [23]:
# Обучаем модель Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [24]:
# Получаем векторы для каждого текста
def get_vector(text):
    words = text.split()
    words = [word for word in words if word in model.wv]
    if len(words) == 0:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)

In [25]:
W2V_X = np.array([get_vector(text) for text in df['link_text_clean']])

In [26]:
print(get_recommendations(movie_titles, df,W2V_X,cosine_classifier))

               Original                         Recommended  Similarity
0   Edward Scissorhands              Dick Tracy (1990 film)    0.991220
1   Edward Scissorhands                      Batman Returns    0.989381
2   Edward Scissorhands                      Ed Wood (film)    0.985612
3   Edward Scissorhands                       Mars Attacks!    0.984992
4   Edward Scissorhands      Planet of the Apes (2001 film)    0.984777
5     La La Land (film)               Moonlight (2016 film)    0.993361
6     La La Land (film)                      Birdman (film)    0.988456
7     La La Land (film)                         Juno (film)    0.988260
8     La La Land (film)                Whiplash (2014 film)    0.987651
9     La La Land (film)                   Midnight in Paris    0.987605
10        Amelia (film)                  The Emerald Forest    0.991433
11        Amelia (film)                   Little Boy (film)    0.989536
12        Amelia (film)                           K2 (film)    0

**Для Word2Vec Средняя Precision@5=((1/5)+(3/5)+(2/5)+(4/5)+(2/5))/5=0,48**

Для уменьшения размерности и дальнейшей визуализации воспользуемся PCA

In [27]:
def apply_pca(X):
    pca = PCA(n_components=2)
    X_embedded = pca.fit_transform(X)
    return X_embedded


In [28]:
# Применяем PCA
W2V_X_embedded = apply_pca(W2V_X)

# Визуализируем кластеры
visualize_clusters(W2V_X_embedded, df)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Данные также явно делятся на два кластера, индийское и западное кино

# BERT

*BERT (Bidirectional Encoder Representations from Transformers) — это языковая модель от Google, обученная с использованием двунаправленного контекста.BERT для эмбеддингов преобразует текст в векторное представление, где каждое слово или предложение кодируется в виде многомерного вектора. Эти эмбеддинги учитывают контекст, в котором слово встречается, благодаря двунаправленной обработке текста.*

Возьмем предобученую модель BERT из библиотеки transformers Hugging Face

In [29]:
from transformers import BertTokenizer, BertModel
import torch

In [30]:
# Загрузка BERT токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [31]:
# Перемещение модели на GPU или оставление на CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [33]:
# Получаем векторы для каждого текста
def get_bert_vector(text):
    # Токенизация с перемещением на нужное устройство
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Перемещение данных на GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Берем среднее значение по всем токенам и перемещаем результат на CPU
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

In [34]:
# Генерируем векторы
X = np.array([get_bert_vector(text).flatten() for text in df['link_text_clean']])

In [35]:
print(get_recommendations(movie_titles, df, X,cosine_classifier))

               Original                        Recommended  Similarity
0   Edward Scissorhands                    Horrible Bosses    0.978903
1   Edward Scissorhands              Into the Woods (film)    0.978654
2   Edward Scissorhands              Beverly Hills Cop III    0.978010
3   Edward Scissorhands                     Ed Wood (film)    0.977793
4   Edward Scissorhands                    Lethal Weapon 2    0.977535
5     La La Land (film)        Fifty Shades of Grey (film)    0.982081
6     La La Land (film)                      Trolls (film)    0.979991
7     La La Land (film)                         Joy (film)    0.979474
8     La La Land (film)                       Carol (film)    0.977784
9     La La Land (film)                       The Hangover    0.977091
10        Amelia (film)                 Lake Placid (film)    0.976416
11        Amelia (film)     The Scarlet Letter (1995 film)    0.975413
12        Amelia (film)   Deliver Us from Evil (2014 film)    0.975199
13    

**Для BERT: Средняя Precision@5=((2/5)+(3/5)+(0/5)+(5/5)+(4/5))/5=0,56**

Для уменьшения размерности и дальнейшей визуализации воспользуемся PCA

In [36]:
# Применяем PCA
Bert_X_embedded = apply_pca(X)

# Визуализируем кластеры
visualize_clusters(Bert_X_embedded, df)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Данные также явно делятся на два кластера, индийское и западное кино

### **Результат: Лучшим векторизатором показал себя TF-IDF(Средняя Precision@5=0,60),далее BERT(Средняя Precision@5=0,56) и последним Word2vec(Средняя Precision@5=0,48)**


---



# Анализ классификаторов (KNN,SVM)

TF-IDF показал наиболее релевантные результаты, за основу возьмем его

# KNN

*К-ближайших соседей (K-Nearest Neighbors или просто KNN) — алгоритм классификации и регрессии, основанный на гипотезе компактности, которая предполагает, что расположенные близко друг к другу объекты в пространстве признаков имеют схожие значения целевой переменной или принадлежат к одному классу*

In [38]:
# Функция для KNN
def knn_classifier(X, idx, n=5):
    knn = NearestNeighbors(n_neighbors=n+1, metric='cosine')
    knn.fit(X)
    distances, indices = knn.kneighbors(X[idx].reshape(1, -1))
    return indices.flatten()[1:], 1 - distances.flatten()[1:]

In [39]:
knn_rec=get_recommendations(movie_titles, df, tf_X,knn_classifier)
print(knn_rec)

               Original                              Recommended  Similarity
0   Edward Scissorhands                           Batman Returns    0.363087
1   Edward Scissorhands                           Ed Wood (film)    0.328485
2   Edward Scissorhands                   Dick Tracy (1990 film)    0.319472
3   Edward Scissorhands           The Nightmare Before Christmas    0.305394
4   Edward Scissorhands           Planet of the Apes (2001 film)    0.295918
5     La La Land (film)             Manchester by the Sea (film)    0.425667
6     La La Land (film)                             Carol (film)    0.374311
7     La La Land (film)                      Chicago (2002 film)    0.362710
8     La La Land (film)                     Whiplash (2014 film)    0.362129
9     La La Land (film)               Lost in Translation (film)    0.358361
10        Amelia (film)                      The Next Karate Kid    0.292141
11        Amelia (film)                              The Reaping    0.291486

**Для TF-IDF + KNN: Средняя Precision@5=((3/5)+(5/5)+(0/5)+(5/5)+(2/5))/5=0,6**

Так как метрикой для KNN мы взяли косинусное расстояние, результаты остались такими же, как когда мы сравнивали косинусное сходство

# SVM

*Метод опорных векторов (англ. support vector machine, SVM) — один из наиболее популярных методов обучения, который применяется для решения задач классификации и регрессии. Основная идея метода заключается в построении гиперплоскости, разделяющей объекты выборки оптимальным способом. Алгоритм работает в предположении, что чем больше расстояние (зазор) между разделяющей гиперплоскостью и объектами разделяемых классов, тем меньше будет средняя ошибка классификатора.*

Будем использовать расстояние от разделяющей гиперплоскости как меру полезности для пользователя

In [40]:
# Функция для создания меток для SVM
def create_labels(df, idx):
    labels = np.zeros(len(df))  # Все фильмы по умолчанию получают метку 0
    labels[idx] = 1  # Текущий фильм получает метку 1
    return labels

# Функция для SVM классификации
def svm_classifier(X, idx, n=5):
    labels = create_labels(df, idx)

    clf = SVC(kernel='linear')
    clf.fit(X, labels)

    distances = clf.decision_function(X)  # Расстояния от гиперплоскости
    similar_indices = distances.argsort()[-n-1:-1][::-1]  # Получаем топ-n

    return similar_indices, distances[similar_indices]


In [41]:
svm_rec=get_recommendations(movie_titles, df, tf_X,svm_classifier)
print(svm_rec)

               Original                     Recommended  Similarity
0   Edward Scissorhands                        Big Fish   -0.999502
1   Edward Scissorhands         Wall Street (1987 film)   -0.999576
2   Edward Scissorhands          Wait Until Dark (film)   -0.999627
3   Edward Scissorhands                       The Brave   -0.999648
4   Edward Scissorhands                   The Lost Boys   -0.999684
5     La La Land (film)           Doctor Strange (film)   -0.999351
6     La La Land (film)           Midnight Rider (film)   -0.999441
7     La La Land (film)                    Sully (film)   -0.999475
8     La La Land (film)           The One I Love (film)   -0.999579
9     La La Land (film)                   The Nice Guys   -0.999628
10        Amelia (film)                Land of the Dead   -0.999571
11        Amelia (film)             Son of a Gun (film)   -0.999584
12        Amelia (film)                Arbitrage (film)   -0.999637
13        Amelia (film)           The Benefactor

**Для TF-IDF + SVM: Средняя Precision@5=((2/5)+(1/5)+(2/5)+(2/5)+(3/5))/5=0,4**

### **Результат:KNN показал лучший результат (Средняя Precision@5=0,6), в отличии от SVM(Средняя Precision@5=0,4)**


---



# Анализ методов усреднения векторов для множественного ввода

Лучшей моделью оказалось TF-IDF+KNN, далее будем использовать ее

In [59]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Обучаем KNN на TF-IDF векторах фильмов
knn = NearestNeighbors(n_neighbors=6, metric='cosine')
knn.fit(tf_X)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [60]:
# Функция для получения вектора фильма
def get_movie_vector(movie_title):
    if movie_title in df['Title'].values:
        movie_idx = df[df['Title'] == movie_title].index[0]
        return tf_X[movie_idx].toarray()[0]  # Преобразуем разреженный формат в плотный
    else:
        return None

Ранее мы искали похожие фильмы, только для одного фильма.Однако, что если на вход попадает несколько фильмов? Какие рекомендации тогда давать пользователю и как получить усредненное представление о вкусе пользователя?

**Постановка эксперимента:** Для проведения этого этапа исследования было опрошено 12 человек, с просьбой назвать 5 любимых фильмов, в результате выдавались рекомендации из 5 фильмов. Полученные рекомендаций,испытуемые оценивали, как посмотрел бы фильм или нет.

Метрикой также возьмем Precision@K,но с другой постановкой Precision@k= количество релевантных (понравившихся) фильмов в первых k рекомендациях/k ,где k=5

А затем возьмем среднее Precision (разделив сумму Precision@K, для каждого испытуемого, на количество испытуемых)

* Важно заметить, что результаты этого эксперимента будет некорректно сравнивать с прошлыми анализами моделей, так как там мы измеряли как алгоритм вычисляет близость фильмов к другому фильму, а здесь вычисляем близость фильмов к вкусу человека, что более абстрактная структура.

## Среднее по векторам

Вычислим среднее из всех полученых векторов, и для среднего найдем ближайшие фильмы

In [61]:
# Функция для рекомендаций на основе нескольких фильмов
def recommend_movies_avg(movie_titles, n=5):
    movie_vectors = [get_movie_vector(title) for title in movie_titles]
    movie_vectors = [vec for vec in movie_vectors if vec is not None]

    if len(movie_vectors) > 0:
        # Вычисляем средний вектор
        avg_vector = np.mean(movie_vectors, axis=0)

        # Ищем фильмы, наиболее близкие к среднему вектору
        distances, indices = knn.kneighbors([avg_vector], n_neighbors=n+len(movie_titles))

        # Убираем из рекомендаций фильмы, которые пользователь уже оценил
        movie_indices = [df[df['Title'] == title].index[0] for title in movie_titles]
        recommendations_indices = [idx for idx in indices[0] if idx not in movie_indices]

        recommendations = df['Title'].iloc[recommendations_indices].values
        return recommendations[:n]  # Ограничиваем количество рекомендаций до n
    else:
        return ["Фильмы не найдены в базе данных."]

In [62]:
# Любимые фильмы экспериментуемых
user_movies = [
    ["Amelia (film)", "Edward Scissorhands", "Shrek", "One Flew Over the Cuckoo's Nest (film)", "La La Land (film)"],
    ["Gladiator (2000 film)", "Schindler's List", "The 300 Spartans", "Limitless (film)", "The Intouchables"],
    ["Harry Potter (film series)", "Rain Man", "The Intouchables", "Titanic (1997 film)", "Black Swan (film)"],
    ["Gone Girl (film)", "The Hunger Games (film series)", "Interstellar (film)", "Shrek", "Titanic (1997 film)"],
    ["Pirates of the Caribbean (film series)", "Lock, Stock and Two Smoking Barrels", "Crank (film)", "John Wick", "Dune (film)"],
    ["Blade Runner", "The Green Mile (film)", "Slumdog Millionaire", "Drive (2011 film)", "Spider-Man (2002 film)"],
    ["A Beautiful Mind (film)", "Pay It Forward (film)", "K-PAX (film)", "The Devil's Advocate (1997 film)", "Constantine (film)"],
    ["Black Swan (film)", "Interstellar (film)", "Snowden (film)", "The Bucket List", "The Princess Diaries (film)"],
    ["The Intern (2015 film)", "Point Break (2015 film)", "The Intouchables", "Spirited Away", "Hidden Figures"],
    ["La La Land (film)", "Harry Potter and the Prisoner of Azkaban (film)", "Harry Potter and the Half-Blood Prince (film)", "Titanic (1997 film)", "Shrek"],
    ["Pride and Prejudice (1940 film)", "The Avengers (2012 film)", "Deadpool (film)", "The Age of Adaline", "Amelia (film)"],
    ["Amelia (film)", "Me Before You (film)", "The Great Gatsby (2013 film)", "Avatar (2009 film)", "Dune (film)"]
]

for movies in user_movies:
    recommended_movies = recommend_movies_avg(movies)
    print(f"\nРекомендации на основе: {', '.join(movies)}")
    print("\n".join(f"{i+1}. {movie}" for i, movie in enumerate(recommended_movies)))



Рекомендации на основе: Amelia (film), Edward Scissorhands, Shrek, One Flew Over the Cuckoo's Nest (film), La La Land (film)
1. Ben-Hur (1959 film)
2. The English Patient (film)
3. Network (film)
4. Gigi (1958 film)
5. Cabaret (1972 film)

Рекомендации на основе: Gladiator (2000 film), Schindler's List, The 300 Spartans, Limitless (film), The Intouchables
1. Amour (2012 film)
2. Gandhi (film)
3. Saving Private Ryan
4. Goodfellas
5. The Killing Fields (film)

Рекомендации на основе: Harry Potter (film series), Rain Man, The Intouchables, Titanic (1997 film), Black Swan (film)
1. Harry Potter and the Philosopher's Stone (film)
2. Gravity (film)
3. Tess (film)
4. Slumdog Millionaire
5. Harry Potter and the Deathly Hallows – Part 2

Рекомендации на основе: Gone Girl (film), The Hunger Games (film series), Interstellar (film), Shrek, Titanic (1997 film)
1. Inception
2. Gravity (film)
3. American Sniper
4. Shrek 2
5. Avatar (2009 film)

Рекомендации на основе: Pirates of the Caribbean (film

**Для TF-IDF + KNN (множественное среднее): Средняя Precision@5=((3/5)+(4/5)+(4/5)+(5/5)+(5/5)+(5/5)+(3/5)+(3/5)+(3/5)+(5/5)+(4/5)+(3/5)/12=0,783**

Хороший результат, однако могут возникнуть следующие проблемы:
* не отражение какой-то темы в рекомендациях, смещение его лишь в одну сторону, как например из-за двух фильмов Гарри-Поттера во входных данных, в рекомендациях получили все остальные фильмы Гарри-Поттера, и совсем не отразили любовь человека к мелодрамам.
* если вкусы у человека слишком разные,будет сложно найти что-то среднее

# Попарные рекомендации

Решить проблемы прошлого метода можно при помощи нахождения похожих фильмов к каждому из любимых фильмов, а не к среднему значению вектора.Таким образом, найдем каждому входному фильму пару.

In [63]:
# Функция для рекомендаций
def recommend_movies_pair(movie_titles):
    recommendations = []

    for title in movie_titles:
        movie_vector = get_movie_vector(title)
        if movie_vector is not None:
            distances, indices = knn.kneighbors([movie_vector], n_neighbors=2)
            # Индекс 0 — это сам фильм, поэтому берем второй ближайший
            closest_idx = indices[0][1]
            recommendations.append(df['Title'].iloc[closest_idx])

    return recommendations

In [64]:
# Любимые фильмы экспериментуемых
user_movies = [
    ["Amelia (film)", "Edward Scissorhands", "Shrek", "One Flew Over the Cuckoo's Nest (film)", "La La Land (film)"],
    ["Gladiator (2000 film)", "Schindler's List", "The 300 Spartans", "Limitless (film)", "The Intouchables"],
    ["Harry Potter (film series)", "Rain Man", "The Intouchables", "Titanic (1997 film)", "Black Swan (film)"],
    ["Gone Girl (film)", "The Hunger Games (film series)", "Interstellar (film)", "Shrek", "Titanic (1997 film)"],
    ["Pirates of the Caribbean (film series)", "Lock, Stock and Two Smoking Barrels", "Crank (film)", "John Wick", "Dune (film)"],
    ["Blade Runner", "The Green Mile (film)", "Slumdog Millionaire", "Drive (2011 film)", "Spider-Man (2002 film)"],
    ["A Beautiful Mind (film)", "Pay It Forward (film)", "K-PAX (film)", "The Devil's Advocate (1997 film)", "Constantine (film)"],
    ["Black Swan (film)", "Interstellar (film)", "Snowden (film)", "The Bucket List", "The Princess Diaries (film)"],
    ["The Intern (2015 film)", "Point Break (2015 film)", "The Intouchables", "Spirited Away", "Hidden Figures"],
    ["La La Land (film)", "Harry Potter and the Prisoner of Azkaban (film)", "Harry Potter and the Half-Blood Prince (film)", "Titanic (1997 film)", "Shrek"],
    ["Pride and Prejudice (1940 film)", "The Avengers (2012 film)", "Deadpool (film)", "The Age of Adaline", "Amelia (film)"],
    ["Amelia (film)", "Me Before You (film)", "The Great Gatsby (2013 film)", "Avatar (2009 film)", "Dune (film)"]
]

for movies in user_movies:
    recommended_movies = recommend_movies_pair(movies)
    print(f"\nРекомендации на основе: {', '.join(movies)}")
    print("\n".join(f"{i+1}. {movie}" for i, movie in enumerate(recommended_movies)))



Рекомендации на основе: Amelia (film), Edward Scissorhands, Shrek, One Flew Over the Cuckoo's Nest (film), La La Land (film)
1. The Next Karate Kid
2. Batman Returns
3. Shrek 2
4. Network (film)
5. Manchester by the Sea (film)

Рекомендации на основе: Gladiator (2000 film), Schindler's List, The 300 Spartans, Limitless (film), The Intouchables
1. The Fall of the Roman Empire (film)
2. The Killing Fields (film)
3. The Four Feathers (1939 film)
4. Freelancers (film)
5. Mustang (film)

Рекомендации на основе: Harry Potter (film series), Rain Man, The Intouchables, Titanic (1997 film), Black Swan (film)
1. Harry Potter and the Goblet of Fire (film)
2. Tootsie
3. Mustang (film)
4. Gigi (1958 film)
5. Shall We Dance (1937 film)

Рекомендации на основе: Gone Girl (film), The Hunger Games (film series), Interstellar (film), Shrek, Titanic (1997 film)
1. The Girl with the Dragon Tattoo (2011 film)
2. The Hungover Games
3. Inception
4. Shrek 2
5. Gigi (1958 film)

Рекомендации на основе: Pirate

**Для TF-IDF + KNN (попарное среднее): Средняя Precision@5=((3/5)+(1/5)+(4/5)+(4/5)+(4/5)+(4/5)+(1/5)+(4/5)+(3/5)+(4/5)+(3/5)+(3/5)/12=0,63**

Проблемы дисбаланса решились, однако:

-Результат Precision ухудшился, это может быть связано с ограниченными условиями вывода(в 5 фильмов),то есть у нас есть всего одна попытка найти пару фильму.Когда для среднего у нас есть 5 попыток найти ближайших к среднему вектору.

-Также рекомендации стали более примитивными.При использовании среднего значения,из-за пересечения каких-то аспектов входных элементов, могли открываться новые,необычные рекомендации.

### **Результат: метод усреднения эмбеддингов имеет более высокий средний Precision@5=0,783, чем попарный метод (средний Precision@5=0,63), а также выдает более глубокие и интересные рекомендации**



---

# Итог

- Лучшим векторизатором показал себя TF-IDF (Средняя Precision@5=0,60),
далее BERT(Средняя Precision@5=0,56) и последним Word2vec (Средняя Precision@5=0,48). Где Precision@k= количество релевантных (похожих) фильмов в первых k рекомендациях/k.А среднее Precision получаем, разделив сумму Precision@K, для каждого фильма выбранного для эксперимента, на количество фильмов, выбранных для эксперимента.

- Применение PCA к векторам ухудшает результаты ранжирования

- Среди классификаторов - KNN показал лучший результат (Средняя Precision@5=0,6), в отличии от SVM (Средняя Precision@5=0,4). Где Precision@k= количество релевантных (похожих) фильмов в первых k рекомендациях/k.А среднее Precision получаем, разделив сумму Precision@K, для каждого фильма выбранного для эксперимента, на количество фильмов, выбранных для эксперимента.

- При множественных входных данных метод усреднения эмбеддингов имеет более высокий средний Precision@5=0,783, чем попарный метод (средний Precision@5=0,63), а также выдает более глубокие и интересные рекомендации. Где Precision@k= количество релевантных (понравившихся) фильмов в первых k рекомендациях/k. А среднее Precision получаем, разделив сумму Precision@K, для каждого испытуемого, на количество испытуемых.